# Pruning with our automatic strutured Pruning framework
Welcome to an end-to-end example for magnitude-based weight pruning

**Summary**

In this tutorial, you will:

* Train a tf.keras model for Cifa10 from scratch.
* Fine tune the model by applying the pruning Framework and see the accuracy.





In [ ]:
!git clone https://github.com/Hahn-Schickard/Automatic-Structured-Pruning
!echo $CWD
!cp -rf /content/Automatic-Structured-Pruning/* /content/

#The Framework work with Tensorflow 2.1-2.3.1
import Pruning

## Train a model for CIFA10 without pruning
Download and prepare the CIFAR10 dataset
The CIFAR10 dataset contains 60,000 color images in 10 classes, with 6,000 images in each class. The dataset is divided into 50,000 training images and 10,000 testing images. The classes are mutually exclusive and there is no overlap between them.

Create the convolutional base
The 6 lines of code below define the convolutional base using a common pattern: a stack of Conv2D and MaxPooling2D layers.

As input, a CNN takes tensors of shape (image_height, image_width, color_channels), ignoring the batch size. If you are new to these dimensions, color_channels refers to (R,G,B). In this example, you will configure our CNN to process inputs of shape (32, 32, 3), which is the format of CIFAR images. You can do this by passing the argument input_shape to our first layer.

To complete our model, you will feed the last output tensor from the convolutional base (of shape (4, 4, 64)) into one or more Dense layers to perform classification. Dense layers take vectors as input (which are 1D), while the current output is a 3D tensor. First, you will flatten (or unroll) the 3D output to 1D, then add one or more Dense layers on top. CIFAR has 10 output classes, so you use a final Dense layer with 10 outputs and a softmax activation.

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
#import matplotlib.pyplot as plt
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

model.summary()



Above, you can see that the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). The width and height dimensions tend to shrink as you go deeper in the network. The number of output channels for each Conv2D layer is controlled by the first argument (e.g., 32 or 64). Typically, as the width and height shrink, you can afford (computationally) to add more output channels in each Conv2D layer.
As you can see, our (4, 4, 64) outputs were flattened into vectors of shape (1024) before going through two Dense layers.


## Compile and train the model

In [ ]:

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
model.save('my_model.h5') 

## Fine-tune pre-trained model with pruning
You will apply pruning to the whole model and see this in the model summary.

In this example, you prune the model with 10% Dense pruning and 10% Filter pruning.



In [ ]:
Denseprunerate=10
Convprunerate=10
import Pruning
prunemodel=Pruning.prune_model('./my_model.h5',Denseprunerate, Convprunerate)

We see how we get less Parameter in the pruned Model

## Compile and re-train the model

In [ ]:
compile = {
  "optimizer": 'adam',
  "loss": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  "metrics": 'accuracy'
}

prunemodel.compile(**compile,)

history = prunemodel.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

test_loss, test_acc = prunemodel.evaluate(test_images,  test_labels, verbose=2)

Compare both Models

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
test_loss, test_acc = prunemodel.evaluate(test_images,  test_labels, verbose=2)

#Prune a model to a maximum accuracy loss

We define the compile arguments and the arguments fpr the Model.fit() function. 

In [ ]:
from keras.callbacks import EarlyStopping
import Pruning

comp = {
  "optimizer": 'adam',
  "loss": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  "metrics": 'accuracy'
}


fit = {
  "epochs": 10,
  "callbacks": EarlyStopping(monitor='val_accuracy', min_delta= 1e-3, mode='min', verbose=1, patience=5, restore_best_weights=True)
}



automodel=Pruning.pruning_for_acc('./my_model.h5',train_images,train_labels,test_images,  test_labels,comp,fit,None,5)